# Глубинное обучение для текстовых данных, ФКН ВШЭ
## Домашнее задание 4: Уменьшение размеров модели
### Оценивание и штрафы

Максимально допустимая оценка за работу — __10 баллов__.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Весь код должен быть написан самостоятельно. Чужим кодом для пользоваться запрещается даже с указанием ссылки на источник. В разумных рамках, конечно. Взять пару очевидных строчек кода для реализации какого-то небольшого функционала можно.

Неэффективная реализация кода может негативно отразиться на оценке. Также оценка может быть снижена за плохо читаемый код и плохо оформленные графики. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

__Мягкий дедлайн 29.11.24 23:59__ \
__Жесткий дедлайн 2.12.24 23:59__

### О задании

В этом задании вам предстоит научиться решать задачу Named Entity Recognition (NER) на самом популярном датасете – [CoNLL-2003](https://paperswithcode.com/dataset/conll-2003). В вашем распоряжении будет предобученный BERT, который вам необходимо уменьшить с минимальными потерями в качестве до размера 20М параметров. Для этого вы самостоятельно реализуете факторизацию эмбеддингов, дистилляцию, шеринг параметров и так далее.

В этом задании вам придется проводить довольно много экспериментов, поэтому мы рекомендуем не писать весь код в тетрадке, а завести разные файлы для отдельных логических блоков и скомпоновать все в виде проекта. Это позволит вашему ноутбуку не разрастаться и сильно облегчит задачу и вам, и проверяющим. Так же постарайтесь логгировать все ваши эксперименты в wandb, чтобы ничего не потерялось.

### Оценивание
Оценка за это домашнее задание будет формироваться из оценки за __задания__ и за __отчет__, в котором, как и раньше, от вас требуется написать о проделанной работе. За отчет можно получить до 2-х баллов, однако в случае отсутствия отчета можно потерять баллы за сами задания. Задания делятся на две части: _номерные_ и _на выбор_. За _номерные_ можно получить в сумме 6 баллов, за задания _на выбор_ можно получить до 16. То есть за все дз можно получить 24 балла. Все, что вы наберете свыше 10, будет считаться бонусами.


### О датасете

Named Entity Recognition – это задача классификации токенов по классам сущностей. В CoNLL-2003 для именования сущностей используется маркировка **BIO** (Beggining, Inside, Outside), в которой метки означают следующее:

- *B-{метка}* – начало сущности *{метка}*
- *I-{метка}* – продолжнение сущности *{метка}*
- *O* – не сущность

Существуют так же и другие способы маркировки, например, BILUO. Почитать о них можно [тут](https://en.wikipedia.org/wiki/Inside–outside–beginning_(tagging)) и [тут](https://www.youtube.com/watch?v=dQw4w9WgXcQ).

Всего в датасете есть 9 разных меток.
- O – слову не соответствует ни одна сущность.
- B-PER/I-PER – слово или набор слов соответстует определенному _человеку_.
- B-ORG/I-ORG – слово или набор слов соответстует определенной _организации_.
- B-LOC/I-LOC – слово или набор слов соответстует определенной _локации_.
- B-MISC/I-MISC – слово или набор слов соответстует сущности, которая не относится ни к одной из предыдущих. Например, национальность, произведение искусства, мероприятие и т.д.

Приступим!

Начнем с загрузки и предобработки датасета.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datasets import load_dataset

dataset = load_dataset("eriktks/conll2003", trust_remote_code=True)

dataset = dataset.remove_columns(["id", "pos_tags", "chunk_tags"])
dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 3453
    })
})

In [ ]:
dataset["train"][0]

{'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [17]:
label_names = [
    "O",
    "B-PER",
    "I-PER",
    "B-ORG",
    "I-ORG",
    "B-LOC",
    "I-LOC",
    "B-MISC",
    "I-MISC",
]

In [ ]:
words = dataset["train"][0]["tokens"]
labels = dataset["train"][0]["ner_tags"]

for i in range(len(words)):
    print(f"{words[i]}\t{label_names[labels[i]]}")

EU	B-ORG
rejects	O
German	B-MISC
call	O
to	O
boycott	O
British	B-MISC
lamb	O
.	O


### Предобработка

На протяжении всего домашнего задания мы будем использовать _cased_ версию BERT, то есть токенизатор будет учитывать регистр слов. Для задачи NER регистр важен, так как имена и названия организаций или предметов искусства часто пишутся с большой буквы, и будет глупо прятать от модели такую информацию.

In [6]:
from transformers import AutoTokenizer, BatchEncoding

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

При токенизации слова могут разделиться на несколько токенов (как слово `Fischler` из примера ниже), из-за чего появится несоответствие между числом токенов и меток. Это несоответствие нам придется устранить вручную.

In [ ]:
example = dataset["train"][12]
words = example["tokens"]
tags = [label_names[t] for t in example["ner_tags"]]
tokenized_text = tokenizer(example["tokens"], is_split_into_words=True)

print("Слова: ", words)
print("Токены:", tokenized_text.tokens())
print("Метки:", tags)

Слова:  ['Only', 'France', 'and', 'Britain', 'backed', 'Fischler', "'s", 'proposal', '.']
Токены: ['[CLS]', 'Only', 'France', 'and', 'Britain', 'backed', 'Fi', '##sch', '##ler', "'", 's', 'proposal', '.', '[SEP]']
Метки: ['O', 'B-LOC', 'O', 'B-LOC', 'O', 'B-PER', 'O', 'O', 'O']


__Задание 1 (1 балл).__ Токенизируйте весь датасет и для каждого текста выравните токены с метками так, чтобы каждому токену соответствовала одна метка. При этом важно сохранить нотацию BIO. И не забудьте про специальные токены! Должно получиться что-то такое:

In [8]:
# aligned_labels = align_labels_with_tokens(example["ner_tags"], tokenized_text)
# tags = [label_names[t] if t > -1 else t for t in aligned_labels]
# print("Выровненные метки:", aligned_labels)
# print("Выровненные названия меток:", tags)

In [ ]:
def align_labels_with_tokens(
    labels: list[int], tokenized_inputs: BatchEncoding, batch_index: int = 0
):
    result = [-100]
    for i in range(1, len(tokenized_inputs.tokens(batch_index)[1:-1]) + 1):
        cur_token_word = tokenized_inputs.token_to_word(batch_index, i)
        prev_token_word = tokenized_inputs.token_to_word(batch_index, i - 1)
        if cur_token_word == prev_token_word:
            if label_names[labels[cur_token_word]][0] == "B":
                result.append(labels[cur_token_word] + 1)
            else:
                result.append(labels[cur_token_word])
        else:
            result.append(labels[cur_token_word])
    result.append(-100)
    return result


tokenized_text.tokens(0), example["ner_tags"], align_labels_with_tokens(
    example["ner_tags"], tokenized_text
)

(['[CLS]',
  'Only',
  'France',
  'and',
  'Britain',
  'backed',
  'Fi',
  '##sch',
  '##ler',
  "'",
  's',
  'proposal',
  '.',
  '[SEP]'],
 [0, 5, 0, 5, 0, 1, 0, 0, 0],
 [-100, 0, 5, 0, 5, 0, 1, 2, 2, 0, 0, 0, 0, -100])

In [10]:
aligned_labels = align_labels_with_tokens(example["ner_tags"], tokenized_text)
tags = [label_names[t] if t > -1 else t for t in aligned_labels]
print("Выровненные метки:", aligned_labels)
print("Выровненные названия меток:", tags)

Выровненные метки: [-100, 0, 5, 0, 5, 0, 1, 2, 2, 0, 0, 0, 0, -100]
Выровненные названия меток: [-100, 'O', 'B-LOC', 'O', 'B-LOC', 'O', 'B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', -100]


In [ ]:
from tqdm import tqdm


def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        my_label_ids = align_labels_with_tokens(label, tokenized_inputs, i)
        labels.append(my_label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


tokenized_dataset_train = dataset["train"].map(tokenize_and_align_labels, batched=True)
tokenized_dataset_val = dataset["validation"].map(
    tokenize_and_align_labels, batched=True
)
tokenized_dataset_test = dataset["test"].map(tokenize_and_align_labels, batched=True)

### Метрика

Для оценки качества NER обычно используют F1 меру с микро-усреднением. Мы загрузим ее из библиотеки `seqeval`. Функция `f1_score` принимает два 2d списка с правильными и предсказанными метками, записаными текстом, и возвращает для них значение F1. Вы можете использовать ее с параметрами по умолчанию.

In [12]:
# ! pip install seqeval

In [11]:
from src import f1, label2id, id2label

Особенность подсчета F1 для NER заключается в том, что в некоторых ситуациях неправильные ответы могут засчитываться как правильные. Например, если модель предсказала `['I-PER', 'I-PER']`, то мы можем догадаться, что на самом деле должно быть `['B-PER', 'I-PER']`, так как сущность не может начинаться с `I-`. Функция `f1_score` учитывает это и поэтому работает только с текстовыми представлениями меток.

### Модель

В качестве базовой модели мы возьмем `bert-base-cased`. Как вы понимаете, он не обучался на задачу NER. Поэтому прежде чем приступать к уменьшению размера BERT, его необходимо дообучить.

__Задание 2 (1 балл)__ Дообучите `bert-base-cased` на нашем датасете с помощью обычного fine-tuning. У вас должно получиться хотя бы 0.9 F1 на тестовой выборке. Заметьте, что чем выше качество большой модели, тем лучше будет работать дистиллированный ученик. Для обучения можно использовать `Trainer` из Hugging Face.

In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    "bert-base-cased", num_labels=len(label_names), id2label=id2label, label2id=label2id
)
print("Число параметров:", sum(p.numel() for p in model.parameters()))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Число параметров: 107726601


In [36]:
from transformers import Trainer, TrainingArguments, DataCollatorForTokenClassification
from accelerate import Accelerator

# accelerator = Accelerator(cpu=True)
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

args = TrainingArguments(
    gradient_accumulation_steps=1,
    auto_find_batch_size=False,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=128,
    torch_compile=False,
    learning_rate=5e-5,
    num_train_epochs=8,
    lr_scheduler_type="cosine",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    logging_first_step=True,
    warmup_steps=200,
    weight_decay=0.5,
    bf16=True,
    output_dir="./results_new",
    max_grad_norm=5,
    metric_for_best_model="f1",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_val,
    compute_metrics=f1,
)

trainer.train()

comet_ml is installed but the Comet API Key is not configured. Please set the `COMET_API_KEY` environment variable to enable Comet logging. Check out the documentation for other ways of configuring it: https://www.comet.com/docs/v2/guides/experiment-management/configure-sdk/#set-the-api-key


Epoch,Training Loss,Validation Loss,F1
1,0.076800,0.071734,0.905218
2,0.045300,0.072461,0.922157
3,0.026400,0.059304,0.927137
4,0.012700,0.065456,0.936825
5,0.005500,0.069959,0.939472
6,0.003800,0.075719,0.937864
7,0.001800,0.079177,0.939755
8,0.001600,0.079302,0.939412


TrainOutput(global_step=3512, training_loss=0.05353578866872054, metrics={'train_runtime': 478.9242, 'train_samples_per_second': 234.542, 'train_steps_per_second': 7.333, 'total_flos': 3113344249179606.0, 'train_loss': 0.05353578866872054, 'epoch': 8.0})

In [38]:
trainer.evaluate(eval_dataset=tokenized_dataset_test)

{'eval_loss': 0.1901179999113083,
 'eval_f1': 0.9012883008356546,
 'eval_runtime': 3.5735,
 'eval_samples_per_second': 966.286,
 'eval_steps_per_second': 7.556,
 'epoch': 8.0}

In [ ]:
# import gc
# import torch

# model = None
# trainer = None
# args = None
# gc.collect()
# torch.cuda.empty_cache()

### Факторизация матрицы эмбеддингов

Можно заметить, что на данный момент матрица эмбеддингов занимает $V \cdot H = 28996 \cdot 768 = 22.268.928$ параметров. Это aж пятая часть от всей модели! Давайте попробуем что-то с этим сделать. В модели [ALBERT](https://arxiv.org/pdf/1909.11942.pdf) предлагается факторизовать матрицу эмбеддингов в произведение двух небольших матриц. Таким образом, параметры эмбеддингов будут содержать $V \cdot E + E \cdot H$ элементов, что гораздо меньше $V \cdot H$, если $H \gg E$. Авторы выбирают $E = 128$, однако ничего не мешает нам взять любое другое значение. Например, выбрав $H = 64$, мы уменьшим число параметров примерно на 20М.

__Задание 3 (1 балл).__ Напишите класс-обертку над слоем эмбеддингов, который реализует факторизацию на две матрицы, и дообучите факторизованную модель. Заметьте, обе матрицы можно инициализировать с помощью SVD разложения, чтобы начальное приближение было хорошим. Это сэкономит очень много времени на дообучении. С рангом разложения $H = 64$ у вас должно получиться F1 больше 0.87.

In [ ]:
from src import FactorizedEmbeddings

factorized_model = AutoModelForTokenClassification.from_pretrained(
    "./results/checkpoint-3512",
    num_labels=len(label_names),
    id2label=id2label,
    label2id=label2id,
)
factorized_model.bert.embeddings.word_embeddings = FactorizedEmbeddings(
    factorized_model.bert.embeddings.word_embeddings.num_embeddings,
    factorized_model.bert.embeddings.word_embeddings.embedding_dim,
    factor=64,
    pretrained_embeds=factorized_model.bert.embeddings.word_embeddings.weight.data,
)
print("Число параметров:", sum(p.numel() for p in factorized_model.parameters()))

Число параметров: 87362569


In [40]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

args = TrainingArguments(
    gradient_accumulation_steps=1,
    auto_find_batch_size=False,
    per_device_train_batch_size=32,
    torch_compile=False,
    learning_rate=3e-5,
    num_train_epochs=8,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    logging_first_step=True,
    warmup_steps=200,
    weight_decay=0.1,
    bf16=True,
    output_dir="./factorized_model",
    max_grad_norm=5,
    metric_for_best_model="f1",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=factorized_model,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_val,
    compute_metrics=f1,
)

trainer.train()

comet_ml is installed but the Comet API Key is not configured. Please set the `COMET_API_KEY` environment variable to enable Comet logging. Check out the documentation for other ways of configuring it: https://www.comet.com/docs/v2/guides/experiment-management/configure-sdk/#set-the-api-key


Epoch,Training Loss,Validation Loss,F1
1,0.080300,0.107726,0.878154
2,0.051400,0.089759,0.909076
3,0.024800,0.092650,0.913833
4,0.014800,0.096394,0.914731
5,0.009700,0.098967,0.921107
6,0.006000,0.098698,0.921502
7,0.006500,0.100313,0.925199
8,0.004100,0.099999,0.927326


TrainOutput(global_step=3512, training_loss=0.039120385191002374, metrics={'train_runtime': 583.5714, 'train_samples_per_second': 192.484, 'train_steps_per_second': 6.018, 'total_flos': 3115143236267478.0, 'train_loss': 0.039120385191002374, 'epoch': 8.0})

In [41]:
trainer.evaluate(eval_dataset=tokenized_dataset_test)

{'eval_loss': 0.24226897954940796,
 'eval_f1': 0.8779044469584892,
 'eval_runtime': 13.4892,
 'eval_samples_per_second': 255.982,
 'eval_steps_per_second': 32.026,
 'epoch': 8.0}

### Дистилляция знаний

Дистилляция знаний – это парадигма обучения, в которой знания модели-учителя дистиллируются в модель-ученика. Учеником может быть произвольная модель меньшего размера, решающая ту же задачу, однако обычно ученик имеет ту же архитектуру, что и учитель. При дистилляции используются два функционала ошибки:

1. Стандартная кросс-энтропия.
1. Функция, задающая расстояние между распределениями предсказаний учителя и ученика. Чаще всего используют KL-дивергенцию.

Для того, чтобы распределение предсказаний учителя не было вырожденным, к softmax добавляют температуру больше 1, например, 2 или 5.   
__Важно:__ при делении логитов на температуру значения градиентов уменьшаются в $\tau^2$ раз (проверьте это!). Поэтому для возвращения их в изначальный масштаб ошибку надо домножить на $\tau^2$. Подробнее об этом можно почитать в разделе 2.1 [оригинальной статьи](https://arxiv.org/pdf/1503.02531).

<img src="https://intellabs.github.io/distiller/imgs/knowledge_distillation.png" width="1000">

__Задание 4 (3 балла).__ Реализуйте метод дистилляции знаний, изображенный на картинке. Для подсчета ошибки между предсказаниями ученика и учителя используйте KL-дивергенцию [`nn.KLDivLoss(reduction="batchmean")`](https://pytorch.org/docs/stable/generated/torch.nn.KLDivLoss.html) (обратите внимание на вормат ее входов). Для получения итоговой ошибки суммируйте мягкую ошибку с жесткой.   
В качестве учителя используйте дообученный BERT из задания 2. В качестве ученика возьмите необученную модель с размером __не больше 20M__ параметров. Вы можете использовать факторизацию матрицы эмбеддингов для уменьшения числа параметров. Если вы все сделали правильно, то на тестовой выборке вы должны получить значение F1 не меньше 0.7. Вам должно хватить примерно 20к итераций обучения для этого. Если у вас что-то не получается, то можно ориентироваться на статью про [DistilBERT](https://arxiv.org/abs/1910.01108) и на [эту статью](https://www.researchgate.net/publication/375758425_Knowledge_Distillation_Scheme_for_Named_Entity_Recognition_Model_Based_on_BERT).

__Важно:__
* Не забывайте добавлять _warmup_ при обучении ученика.
* Не забывайте переводить учителя в режим _eval_.

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
    "./results_new/checkpoint-3512",
    num_labels=len(label_names),
    id2label=id2label,
    label2id=label2id,
)

In [19]:
from src import DistilTrainer

In [ ]:
from transformers import (
    BertConfig,
    BertForTokenClassification,
    Trainer,
    TrainingArguments,
    DataCollatorForTokenClassification,
)

config = BertConfig(
    vocab_size=model.config.vocab_size,
    hidden_size=306,
    num_hidden_layers=9,
    num_attention_heads=6,
    intermediate_size=1312,
    num_labels=len(label_names),
)
student_model = BertForTokenClassification(
    config,  # num_labels=len(label_names), id2label=id2label, label2id=label2id
)
# student_model.bert.embeddings.word_embeddings = FactorizedEmbeddings(
#     config.vocab_size,
#     config.hidden_size,
#     factor=354,
#     pretrained_embeds=None # model.bert.embeddings.word_embeddings.weight,
# )

# student_model.classifier = nn.Linear(config.hidden_size, len(label_names))
print("Число параметров:", sum(p.numel() for p in student_model.parameters()))

Число параметров: 19667421


In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

args = TrainingArguments(
    gradient_accumulation_steps=1,
    auto_find_batch_size=False,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=128,
    torch_compile=False,
    learning_rate=3e-4,
    max_steps=15_000,
    lr_scheduler_type="cosine",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    logging_first_step=True,
    warmup_steps=500,
    weight_decay=0.01,
    bf16=True,
    output_dir="./distilled_model",
    max_grad_norm=5,
    metric_for_best_model="f1",
    load_best_model_at_end=True,
)

trainer = DistilTrainer(
    teacher_model=model,
    student_model=student_model,
    temperature=6,
    lambda_param=0.7,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_val,
    compute_metrics=f1,
)

trainer.train()

comet_ml is installed but the Comet API Key is not configured. Please set the `COMET_API_KEY` environment variable to enable Comet logging. Check out the documentation for other ways of configuring it: https://www.comet.com/docs/v2/guides/experiment-management/configure-sdk/#set-the-api-key
max_steps is given, it will override any value given in num_train_epochs


Epoch,Training Loss,Validation Loss,F1
1,75.339500,88.820152,0.374151
2,47.287500,60.118763,0.523584
3,34.392100,52.367016,0.592548
4,25.526300,43.998981,0.671033
5,22.070700,42.231491,0.709986
6,19.228600,40.230412,0.704988
7,18.789500,39.654247,0.712349
8,13.600600,37.654114,0.743718
9,12.771600,38.105419,0.737823
10,9.683300,35.405483,0.756686


TrainOutput(global_step=15000, training_loss=14.03398786315918, metrics={'train_runtime': 2170.6218, 'train_samples_per_second': 221.135, 'train_steps_per_second': 6.91, 'total_flos': 1663835374662300.0, 'train_loss': 14.03398786315918, 'epoch': 34.16856492027335})

In [54]:
trainer.evaluate(eval_dataset=tokenized_dataset_test)

{'eval_loss': 38.551422119140625,
 'eval_f1': 0.7327578690382771,
 'eval_runtime': 9.5932,
 'eval_samples_per_second': 359.943,
 'eval_steps_per_second': 2.814,
 'epoch': 34.16856492027335}

# Задания на выбор

Как вы понимаете, есть еще довольно много разных способов уменьшить обученную модель. В этой секции вам предлагается реализовать разные техники на выбор. За каждую из них можно получить разное количество балов в зависимости от сложности. Успешность реализации будет оцениваться как по коду, так и по качеству на тестовой выборке. Все баллы за это дз, которые вы наберете сверх 10, будут считаться бонусными.   
В задании 4 вы обучали модель с ограничением числа параметров в 20М. При реализации техник из этой секции придерживайтесь такого же ограничение. Это позволит честно сравнивать методы между собой и делать правильные выводы. Напишите в отчете обо всем, что вы попробовали.

* __Шеринг весов (2 балла).__ В модификации BERT [ALBERT](https://arxiv.org/pdf/1909.11942.pdf) помимо факторизации эмбеддингов предлагается шерить веса между слоями. То есть разные слои используют одни и те же веса. Такая техника эвивалентна применению одного и того же слоя несколько раз. Она позволяет в несколько раз уменьшить число параметров и не сильно потерять в качестве.
* __Факторизация промежуточных слоев (2 балла).__ Если можно факторизовать матрицу эмбеддингов, то и все остальное тоже можно. Для факторизации слоев существует много разных подходов и выбрать какой-то один сложно. Вы можете вдохновляться [этим списком](https://lechnowak.com/posts/neural-network-low-rank-factorization-techniques/), найти в интернете что-то другое или придумать метод самостоятельно. В любом случае в отчете обоснуйте, почему вы решили сделать так как сделали.
* __Приближение промежуточных слоев (2 балла).__ Мы обсуждали, что помимо приближения выходов модели ученика к выходам модели учителя, можно приближать выходы промежуточных слоев. В [этой работе](https://www.researchgate.net/publication/375758425_Knowledge_Distillation_Scheme_for_Named_Entity_Recognition_Model_Based_on_BERT) подробно написано, как это можно сделать.
* __Прунинг (4 балла).__ В методе [SparseGPT](https://arxiv.org/abs/2301.00774) предлагается подход, удаляющий веса модели один раз после обучения. При этом оказывается возможным удалить до половины всех весов без потери в качестве. Математика, стоящаяя за техникой, довольно сложная, однако общий подход простой – будем удалять веса в каждом слое по отдельности, при удалении части весов слоя, остальные веса будут перенастраиваться так, чтобы общий выход слоя не изменился.
* __Удаление голов (6 баллов).__ В данный момент мы используем все головы внимания, но ряд исследований показывает, что большинство из них можно выбросить без потери качества. В этой [статье](https://arxiv.org/pdf/1905.09418.pdf) предлагается подход, который добавляет гейты к механизму внимания, которые регулируют, какие головы участвуют в слое, а какие – нет. В процессе обучения гейты настраиваются так, чтобы большинство голов не использовалась. В конце обучения неиспользуемые головы можно удалить. За это задание дается много баллов, потому что в методе довольно сложная математика и подход плохо заводится. Если вы решитесь потратить на него свои силы, то в случае неудачи мы дадим промежуточные баллы, опираясь на отчет.   
__Совет:__ во время обучения внимательно следите за поведением гейтов. Если вы все сделали правильно, то они должны зануляться. Однако зануляются они не всегда сразу, им надо дать время и обучать модель подольше.

In [ ]:
# weight sharing - training with embedding factorizations works awfully bad!
from torch import nn

weight_sharing_config = BertConfig(
    vocab_size=model.config.vocab_size,
    hidden_size=552,
    num_hidden_layers=12,
    num_attention_heads=8,
    intermediate_size=2208,
    num_labels=len(label_names),
)
weight_sharing_model = BertForTokenClassification(weight_sharing_config)

layer = weight_sharing_model.bert.encoder.layer[0]
weight_sharing_model.bert.encoder.layer = nn.ModuleList([layer for _ in range(12)])

print("Число параметров:", sum(p.numel() for p in weight_sharing_model.parameters()))

Число параметров: 19959225


In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

args = TrainingArguments(
    gradient_accumulation_steps=1,
    auto_find_batch_size=False,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=128,
    torch_compile=False,
    learning_rate=3e-5,
    max_steps=15_000,
    lr_scheduler_type="cosine",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    logging_first_step=True,
    warmup_steps=500,
    weight_decay=0.01,
    bf16=True,
    output_dir="./weight_sharing_model",
    max_grad_norm=5,
    metric_for_best_model="f1",
    load_best_model_at_end=True,
    save_safetensors=False,
)

trainer = DistilTrainer(
    teacher_model=model,
    student_model=weight_sharing_model,
    temperature=6,
    lambda_param=0.7,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_val,
    compute_metrics=f1,
)

trainer.train()

comet_ml is installed but the Comet API Key is not configured. Please set the `COMET_API_KEY` environment variable to enable Comet logging. Check out the documentation for other ways of configuring it: https://www.comet.com/docs/v2/guides/experiment-management/configure-sdk/#set-the-api-key
max_steps is given, it will override any value given in num_train_epochs


Epoch,Training Loss,Validation Loss,F1
1,124.833100,144.435425,0.055291
2,81.566800,98.556602,0.234211
3,63.743900,76.515282,0.366243
4,48.524800,66.062767,0.486871
5,42.427000,58.565804,0.527373
6,36.168700,55.090431,0.581807
7,37.537800,51.366894,0.592581
8,27.964700,48.028980,0.619902
9,25.779900,47.466835,0.622388
10,20.725500,45.041756,0.652837


TrainOutput(global_step=15000, training_loss=26.165431801350913, metrics={'train_runtime': 2514.8351, 'train_samples_per_second': 190.867, 'train_steps_per_second': 5.965, 'total_flos': 573994338781500.0, 'train_loss': 26.165431801350913, 'epoch': 34.16856492027335})

In [92]:
trainer.evaluate(eval_dataset=tokenized_dataset_test)

{'eval_loss': 42.566070556640625,
 'eval_f1': 0.6674538247448765,
 'eval_runtime': 10.8059,
 'eval_samples_per_second': 319.547,
 'eval_steps_per_second': 2.499,
 'epoch': 34.16856492027335}

In [32]:
# intermediate layer approx

from src import DistilInterTrainer

In [201]:
interlayer_approx_config = BertConfig(
    vocab_size=model.config.vocab_size,
    hidden_size=372,
    num_hidden_layers=model.config.num_hidden_layers // 2,
    num_attention_heads=model.config.num_attention_heads,
    intermediate_size=1200,
    num_labels=len(label_names),
)


class InterLayerBert(BertForTokenClassification):
    def __init__(self, config, teacher_hidn_size: int = 768):
        super().__init__(config)
        self.config = config
        self.hidn_upscale = nn.ModuleList(
            [
                nn.Linear(config.hidden_size, teacher_hidn_size)
                for _ in range(config.num_hidden_layers)
            ]
        )


interlayer_approx_model = InterLayerBert(
    interlayer_approx_config, teacher_hidn_size=768
)

print(
    "Число параметров:",
    sum(p.numel() for p in interlayer_approx_model.parameters())
    - sum(p.numel() for p in interlayer_approx_model.hidn_upscale.parameters()),    # these params are purely for loss calculations
)

Число параметров: 19687125


In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

args = TrainingArguments(
    gradient_accumulation_steps=1,
    auto_find_batch_size=False,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=128,
    torch_compile=False,
    learning_rate=3e-5,
    max_steps=15_000,
    lr_scheduler_type="cosine",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    logging_first_step=True,
    warmup_steps=500,
    weight_decay=0.01,
    bf16=True,
    output_dir="./weight_sharing_model",
    max_grad_norm=5,
    metric_for_best_model="f1",
    load_best_model_at_end=True,
    save_safetensors=False,
)

trainer = DistilInterTrainer(
    teacher_model=model,
    student_model=interlayer_approx_model,
    temperature=6,
    lambda_param=0.7,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_val,
    compute_metrics=f1,
)

trainer.train()

comet_ml is installed but the Comet API Key is not configured. Please set the `COMET_API_KEY` environment variable to enable Comet logging. Check out the documentation for other ways of configuring it: https://www.comet.com/docs/v2/guides/experiment-management/configure-sdk/#set-the-api-key


max_steps is given, it will override any value given in num_train_epochs


Epoch,Training Loss,Validation Loss,F1
1,144.125700,154.632645,0.034062
2,88.592000,109.674675,0.237467
3,71.205400,86.432785,0.352439
4,55.499300,75.955772,0.445298
5,49.626600,66.811913,0.482609
6,43.812200,62.161835,0.520404
7,45.156200,59.746578,0.533967
8,33.666800,55.520119,0.553528
9,31.392600,54.560898,0.566750
10,26.407800,51.151665,0.583378


TrainOutput(global_step=15000, training_loss=33.69489500732422, metrics={'train_runtime': 1997.1911, 'train_samples_per_second': 240.338, 'train_steps_per_second': 7.511, 'total_flos': 1631117882702700.0, 'train_loss': 33.69489500732422, 'epoch': 34.16856492027335})